In [15]:
import glob
import json

import numpy as np
from PIL import Image

In [16]:
with open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/traindatasets/trainset.txt') as file:
    train = file.readlines()

with open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/testdatasets/testset.txt') as file:
    test = file.readlines()

In [17]:
train = [tmp.strip().split('.')[0][5:] for tmp in train]
test = [tmp.strip().split('.')[0][5:] for tmp in test]

In [18]:
mask = np.array(Image.open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_fundusmask/fmask.tif').getchannel(0))

In [19]:
gt = {
'he': sorted(glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/hardexudates/*.png')),
'se': sorted(glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/softexudates/*.png')),
'rsd': sorted(glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/redsmalldots/*.png')),
'hr': sorted(glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/hemorrhages/*.png'))}

In [20]:
combinations = [['he'], ['se'], ['rsd'], ['hr'],
                ['he', 'se'], ['he', 'rsd'], ['he', 'hr'], ['se', 'rsd'], ['se', 'hr'], ['rsd', 'hr'],
                ['he', 'se', 'rsd'], ['he', 'se', 'hr'], ['he', 'rsd', 'hr'], ['se', 'rsd', 'hr'],
                ['he', 'se', 'rsd', 'hr']]

In [21]:
distributions = {}

In [22]:
for comb in combinations:
    for imgs in zip(*[gt[com] for com in comb], strict=False):
        id = imgs[0].split('/')[-1].split('.')[0][-3:]

        if id not in distributions:
            distributions[id] = {}

        images = np.array([np.array(Image.open(img).getchannel(0)) for img in imgs])
        image = np.max(images, axis=0)
        counts_no_fov = np.unique(image, return_counts=True)
        counts_fov = np.unique(image[mask], return_counts=True)

        distributions[id]['-'.join(sorted(comb))] = {'fov': {'values': counts_fov[0].tolist(), 'counts': counts_fov[1].tolist()},
                                'all': {'values': counts_no_fov[0].tolist(), 'counts': counts_no_fov[0].tolist()}}
        print(comb, imgs[0].split('/')[-1], counts_no_fov)

['he'] image001.png (array([  0,  63, 126, 189, 252], dtype=uint8), array([1650808,   42319,    4907,    8823,   21143]))
['he'] image002.png (array([  0,  63, 126, 168, 189], dtype=uint8), array([1647107,   55227,   18777,     269,    6620]))
['he'] image003.png (array([  0,  21,  42,  63,  84, 126, 168, 189], dtype=uint8), array([1656611,    1729,   13799,   31741,    1494,   15110,    2051,
          5465]))
['he'] image004.png (array([  0,  42,  63,  84, 105, 126, 147, 168, 189, 231, 252], dtype=uint8), array([1437683,    2137,  135224,      67,     944,   37323,    1634,
           177,   41163,    1746,   69902]))
['he'] image005.png (array([  0,  42,  63, 105, 126, 189, 252], dtype=uint8), array([1113091,     233,  205292,    6029,  146303,  125047,  132005]))
['he'] image006.png (array([  0,  42,  63, 105, 126, 189, 252], dtype=uint8), array([1648179,    6605,    6633,      33,   18820,   33136,   14594]))


['he'] image007.png (array([  0,  42,  63, 105, 126, 147, 189, 252], dtype=uint8), array([1445768,    9687,   59747,    1660,   39696,      33,   39415,
        131994]))
['he'] image008.png (array([  0,  63, 126, 189, 252], dtype=uint8), array([1625708,   21410,   28668,   22610,   29604]))
['he'] image009.png (array([  0,  21,  63,  84, 105, 126, 168, 189], dtype=uint8), array([1627681,    2143,   42626,    1315,      16,   23797,    4326,
         26096]))
['he'] image010.png (array([  0,  21,  42,  63,  84, 105, 126, 147, 168, 189, 252], dtype=uint8), array([1628710,    1049,    2475,   71039,    2594,    3480,    7089,
           291,    1064,    3590,    6619]))
['he'] image011.png (array([  0,  42,  63, 105, 126, 168, 189, 231, 252], dtype=uint8), array([1596263,    7790,   38419,    3070,   28392,    2966,   21643,
          1014,   28443]))
['he'] image012.png (array([  0,  42,  63,  84, 105, 126, 168, 189, 252], dtype=uint8), array([1663986,   15875,   22745,      33,    1269

In [23]:
distributions

{'001': {'he': {'fov': {'values': [0, 63, 126, 189, 252],
    'counts': [1330043, 42319, 4907, 8823, 21143]},
   'all': {'values': [0, 63, 126, 189, 252],
    'counts': [0, 63, 126, 189, 252]}},
  'se': {'fov': {'values': [0], 'counts': [1407235]},
   'all': {'values': [0], 'counts': [0]}},
  'rsd': {'fov': {'values': [0, 21, 42, 63, 105, 126, 189],
    'counts': [1400907, 33, 5266, 839, 122, 44, 24]},
   'all': {'values': [0, 21, 42, 63, 105, 126, 189],
    'counts': [0, 21, 42, 63, 105, 126, 189]}},
  'hr': {'fov': {'values': [0, 63, 126, 189, 252],
    'counts': [1394356, 7585, 3568, 1693, 33]},
   'all': {'values': [0, 63, 126, 189, 252],
    'counts': [0, 63, 126, 189, 252]}},
  'he-se': {'fov': {'values': [0, 63, 126, 189, 252],
    'counts': [1330043, 42319, 4907, 8823, 21143]},
   'all': {'values': [0, 63, 126, 189, 252],
    'counts': [0, 63, 126, 189, 252]}},
  'he-rsd': {'fov': {'values': [0, 21, 42, 63, 105, 126, 189, 252],
    'counts': [1324331, 33, 5200, 42650, 122, 4933

In [24]:
citation = """
@inproceedings{diaretdb1,
  title={The DIARETDB1 Diabetic Retinopathy Database and Evaluation Protocol},
  author={Tomi Kauppi and Valentina Kalesnykiene and J. K{\"a}m{\"a}r{\"a}inen and Lasse Lensu and Iiris Sorri and A. Raninen and R. Voutilainen and Hannu Uusitalo and Heikki K{\"a}lvi{\"a}inen and Juhani Pietil{\"a}},
  booktitle={British Machine Vision Conference},
  year={2007},
  url={https://api.semanticscholar.org/CorpusID:15483141}
}"""

In [25]:
result = {'distributions': distributions,
          'train': train,
          'test': test,
          'citation': citation}

In [26]:
with open('diaretdb1.json', 'w') as file:
    json.dump(result, file)
